# Web Scraping: Michigan Data and Scraping PDFs

Over the last few weeks, you learned some of the basics of web scraping:

 * Fetching HTML documents and parsing information within the documents using [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
 * Using tools such [Selector Gadget](http://selectorgadget.com/) and [Web Scraper](http://webscraper.io/) to help find (and scrape) elements of web pages
 * Scraping simple HTML tables using [Google Sheets](https://support.google.com/docs/answer/3093339?hl=en)
 * Using the browser's Developer Tools to see if a site is making API calls "behind the scenes" to populate data on the page (this is very common in many modern sites and is usually where we find good scraping data sources)

Today we're practice some more scraping by a) going to go over the Scraping Challenge Round &#x1f3c6; and b) scraping a few Michigan Covid-related sites. Then, we'll learn how to scrape information from PDF files, using tools such as [PDF Plumber](https://github.com/jsvine/pdfplumber). After this, you'll have the scraping skills of something like this:

![Heavy Duty Scraping](http://www.romeplow.com/assets/Uploads/LL-EMscrapers/RP-180HDE_heavy_duty_ejector_scraper.jpg)

## Let's revisit the Challenge Round  &#x1f3c6;

For homework, we had you pick one or more of the following scraping exercises. Let's go over them now! We'll go over the first one in detail and can try the others if anyone has questions on how to approach/solve them:

Site: [https://analytics.usa.gov/](https://analytics.usa.gov/)<br>
Task: Number of people visiting US Government web sites now<br>

~Site: [http://www.state.gov/r/pa/ode/socialmedia/](http://www.state.gov/r/pa/ode/socialmedia/)~<br>
~Task: The number of Pinterest accounts maintained by U.S. State Department embassies and missions~<br>

Site: [https://petitions.whitehouse.gov/](https://petitions.whitehouse.gov/)<br>
Task: Number of petitions that have reached their goal<br>

Site: [https://www.faa.gov/air_traffic/flight_info/aeronav/aero_data/](https://www.faa.gov/air_traffic/flight_info/aeronav/aero_data/)<br>
Task: Number of airports with existing construction related activity<br>

Site: [https://www.osha.gov/pls/imis/establishment.html](https://www.osha.gov/pls/imis/establishment.html)<br>
Number of OSHA enforcement inspections involving Wal-Mart in California since 2014<br>

Site: [https://www.tdcj.state.tx.us/death_row/dr_scheduled_executions.html](https://www.tdcj.state.tx.us/death_row/dr_scheduled_executions.html)<br>
Task: Number of days until Texas's next scheduled execution <br>

### Challenge 1: # of people visiting US Government web sites

>Site: [https://analytics.usa.gov/](https://analytics.usa.gov/)<br>
Task: Number of people visiting US Government web sites now<br><br>

This one is a little "tricky" - if you fetch the page (using requests) and parse it (using BeautifulSoup), you'll notice that the count (number of people) isn't there. 

Let's use `Chrome` to open up [https://analytics.usa.gov/](https://analytics.usa.gov/) and take a look at the HTML of the page. Once you have the page open in Chrome, go to:

`View` --> `Developer` --> `Inspect Elements`. 

After the inspector window opens, use your mouse to hover over the number of visitors (the inspector should highlight the page of the page that your mouse is hovering over) and then click on the numbere. The inspector should highlight the HTML - it will look like this (your number will certainly vary):

```html
<h2 id="current_visitors" class="data">294,883</h2>
```

This is the HTML element that we want - it has the total number of visitors on US Goverment sites at this moment. 

**To parse this element from the page using BeautifulSoup**, we'd do something like this:

In [ ]:
from requests import get
from bs4 import BeautifulSoup

url = 'https://analytics.usa.gov/'

response = get(url)

soup = BeautifulSoup(response.text, 'html.parser')

# we're looking for this part of the page:
# <h2 id="current_visitors" class="data">294,883</h2>

print(soup.find('h2', {'id': 'current_visitors'}))

`...` is not what we're looking for! Where is the number that we saw in the Inspector? This is a clue that the number of visitors is not in the HTML when the page loads in the browser. Let's go back to the Chrome Developer Tools and see if the page is making API calls "behind the scenes" to fetch data for the page. 

Open up the Developer Tools -- you can select `Inspect Elements` or `JavaScript Console`. **Then** select the `Network` tab, select `XHR` and refresh the page. You should see something like this:

![Network Inspector](https://github.com/computationaljournalism/columbia2020/raw/master/images/chrome_network_xhr.png)

**XHR** stands for **X**ML **H**TTP **R**equest. Another name for it is [AJAX](https://en.wikipedia.org/wiki/Ajax_(programming)) or "Asychronous JavaScript and XML." Both of these names/acronyms came from a time when [XML](https://en.wikipedia.org/wiki/XML) was the data format of choice but has *mostly* been replaced by JSON. XHR and AJAX are acronyms for how a website asynchronously fetches (or sends) data to a server. In this case, the usa.gov site is fetching the count of realtime users from a server by making an API call "in the background" while the browser is loading and rendering the page. It happens so quickly that you *shouldn't* even notice that the browser swaps out the `...` for the actual count.

If you're curious, you can read up on to use Chrome's network devloper tools [here](https://developers.google.com/web/tools/chrome-devtools/network/reference)

**So, back to our example!** Click on the `realtime.json` file in the Network tab and take a look. If you look at the `Headers` tab, you'll notice that the URL is [https://analytics.usa.gov/data/live/realtime.json](`https://analytics.usa.gov/data/live/realtime.json`)

Go ahead and open it up in your browser and you'll see that it returns something that looks like the following:

```json
{
  "name": "realtime",
  "sampling": {},
  "query": {
    "metrics": [
      "rt:activeUsers"
    ],
    "max-results": 10000
  },
  "meta": {
    "name": "Active Users Right Now",
    "description": "Number of users currently visiting all sites."
  },
  "data": [
    {
      "active_visitors": "294483"
    }
  ],
  "totals": {},
  "taken_at": "2020-04-12T19:31:41.404Z"
}```

**It's JSON!** 🎉 🎉 We love JSON, right? If you look closely, you'll see that it has a field called `active_visitors` and, when I created this notebook, the count was `294483`. Our scraping challenge just got a lot easier. Instead of having to sift through some HTML to find our count, we can simply call this endpoint to get the count. We are pros at making HTTP request and parsing JSON, right? Let's do this now! Eventhough some (most?) of you already did this challenge, it doesn't hurt to do it again. Write the few lines of code to extract the activie visitors count from `realtime.json`:

In [ ]:
# your turn....fetch realtime.json and extract the number of active visitors





**🔑 Important: Always check the Network (XHR) Tab 🔑**

As you saw in this example, the data we were looking for wasn't in the page HTML but came from an API call. This is true in many cases when you need to scrape a website! So, as you set out to scrape a website, make sure you open the Network (XHR) tab and refresh the page - you may find that the site is fetching the data via an API. In many cases, it's much easier to call the API directly than trying to scrape the information from the HTML.

### Any questions about the other challenges?

Does anyone have questions about the other challenges? Did we get to all of them? We can go over any of them here or can do one together as a class! How about scraping the state date on [http://covidtracking.com/](http://covidtracking.com/)? Did anyone make progress on scraping Michigan data? We'll work on this as a class in a few minutes...


## Scraping Michigan

We met with [Anjanette](https://twitter.com/anjdelgado?lang=en) and [Kristi](https://twitter.com/midatalove) from the [Detroit Free Press](https://www.freep.com/) last week and talked about how we migth help them with data+scraping+reporting on Covid. Let's spend some time in class tonight looking at Michigan data - we can use the time to continue learning some scraping techniques while we start to understand some of the covid and census data from Michigan. 

Let's get started by looking at some site/data that Anjanette sent last week. Here is the state-level data for Covid cases, deaths, tests, etc: https://www.michigan.gov/coronavirus/0,9753,7-406-98163_98173---,00.html

After taking a look at the data, let's look at the page itself and discuss approaches to scraping the data. Open up your "Browser Tools" and take a look at the HTML of the page. As we talked about earlier, look at the Network tab and see if it's loading any data via API calls.

**What approach would you take to scrape the County-level cases on the page?**

(leaving some intentional space here so you don't look ahead :-)

Since the charts/tables on the page (as of 4/13/2020) are all HTML tables, we can use pandas [read_html](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_html.html) to load and parse the tables. Let's do this now as a class.

Please do the following:

1. use `read_html` from pandas to load the Covid data from [this page](https://www.michigan.gov/coronavirus/0,9753,7-406-98163_98173---,00.html),

2. tell us how many tables you were able to find on the page

3. sort the "Overall Confirmed COVID-19 Cases" table by the number of cases (in descending order).

Ready? Go! 

In [ ]:
# your code goes here

# i'll get you started with this:
from pandas import read_html






Once you start to look at the cases table, you'll notice it has mostly County level data, with a few additions:

- Totals
- Detroit City (which is in Wayne County)
- MDOC = Michigan Department of Corrections
- FCI = Federal Correctional Institute
- Unknown
- Out of State

You can easily drop a row in your dataframe, say if you want to remove the "Totals" row. In this sample code, we assume the "Totals" row is in dataframe and has an index of 79 - we can remove it with the following code. Note that we're saving the updated DataFrame to a variable called `cases_df`.

```python
tables = read_html(url)
# let's remove row with index 79
cases_df = tables[0].drop(79)
```

In [ ]:
# make sure you have the covid cases data frame in a variable called cases_df
# this is what we'll use in the code below...
# do this now if you haven't already....




At this point, we should have a dataframe that has all of the Michigan covid cases. Let's look at combining it with some other data that will give us more insight into each Michigan County. By doing this, we'll learn more about each county and some of it's characteristics. It certianly doesn't mean we'll find the causes but we'll learn how to merge a few datasets and start to make some progress on data collection for our friends at the Detroit Free Press.

There are a lot of different datasets we wanted to look at: 

- [unemployment data (Dept of Labor)](https://oui.doleta.gov/unemploy/claims.asp)
- [job types and wages (BLS)](https://data.bls.gov/cew/apps/data_views/data_views.htm#tab=Tables)
- [occupation, commuting, age, sex, race (Census)](https://www.census.gov/acs/www/data/data-tables-and-tools/data-profiles/)

Let's start by looking at the Census data.

## Census Data

A few months ago (way back!), we looked at the Census API. Does anyone remember that (it was a simpler time)?

We started out by making an API call to get county level data for Iowa. Let's modify that code to get the county names, FIPS codes and population for Michigan counties. We need to call the API with the Michigan's FIPS state code and a simple google search shows that Michigan's FIPS code is `26`.

After digging over the Census API docs and looking back over notebook 04, we can create a URL that will return the Michigan county level population data that we want:

https://api.census.gov/data/2018/acs/acs5?get=NAME,B01001_001E&for=county:*&in=state:26

If we breakdown the URL into its various compontents, you'll see that we have the base URL of:

https://api.census.gov/data/2018/acs/acs5

This means we are asking for the ACS 5-year data from 2018.

The URL is then followed by some URL parameters:

  * `get=NAME,B01001_001E`: this tells the API that we want to get the location/geo name and the population specified by this code: B01001_001E. This code comes from a [huge list of "variables"](https://api.census.gov/data/2018/acs/acs5/variables.html) which we can ask for.
  * `for=county:*`: this tells the API that we want every county. The `*`, or wildcard, means "all" in this case.
  * `in=state:26`: this tells the API that we only want data for state #26. The census uses [FIPS codes](https://en.wikipedia.org/wiki/Federal_Information_Processing_Standard_state_code) and 26 is the state code for Michigan.

Instead of calling the Census API with the full url directy, we can tidy up our code by passing the URL and URL parameters (as a dictionary) to the `get()` method in the requests library. It will do the work to "stitch" the two together to look like this:

https://api.census.gov/data/2018/acs/acs5?get=NAME,B01001_001E&for=county:*&in=state:26

In [ ]:
from requests import get

# the census API URL
url = "https://api.census.gov/data/2018/acs/acs5"

# our URL parameters
# here we are saying we want the NAME and B01001_001E (which means the population)
# for each county (ccocunty:*)
# in state number 26 (ie Michigan)
url_parameters = {
    "get": "NAME,B01001_001E",
    "for": "county:*",
    "in": "state:26"
}

# make the api call - the requests library will "stitch" together the URL and URL parameters
response = get(url, params=url_parameters)

data = response.json()

# what type of data did we get?
print(type(data))


And, we can load that into a DataFrame!

In [ ]:
from pandas import DataFrame

michigan_population_df = DataFrame(data)

In [ ]:
michigan_population_df.head()

Well, that doesn't quite look right! The first row in the DataFrame is the "header" row. There are a few ways to fix this, but one approach is to say we want to load everything from `data` but skip the first row. This is done by specifying `data[1:]`, which means "give me everything from the second position in the list, all the way to the end." As part of creating the DataFrame, we can also specify the headers:

In [ ]:
michigan_population_df = pd.DataFrame(data[1:], columns=['CountyName', 'Population', 'StateFIPS', 'CountyFIPS'])

michigan_population_df.head()

Before merging this data with our Covid-county level data frame, let's do a little cleanup:

In [ ]:
# first, let's sort the counties by population
michigan_population_df.sort_values(['Population'])

Hmm... that doesn't like quite right. It looks like it's sorting alphabetically instead of numerically. This means that the data in our Population column of type `str`, not `int` (integer). We can easily convert all of the values in the `Population` column to integers using the following: 

In [ ]:
# convert all values in the Population column to integers and replace them in the data frame
michigan_population_df['Population'] = michigan_population_df['Population'].astype(int)

# then, let's try sorting by population again (descending this time)
michigan_population_df.sort_values(['Population'], ascending=False)

Ok, much better! Now, let's combine it with our Michigan county covid data frame. If you didn't get it loaded above, this should work:

In [ ]:
from pandas import read_html

tables = read_html('https://www.michigan.gov/coronavirus/0,9753,7-406-98163_98173---,00.html')

cases_df = tables[0]

cases_df.head()

### Let's Merge the Two Data Frames

Let's merge the two data frames on the County name, but you'll notice that our cases data frame looks like `Allegan` while our Census data frame has `Allegan County, Michigan`. Let's get these two columns to match so our merge is easier. Here's a quick hack to create a new column in our Cases data frame which creates a new column, adding " County, Michigan" to the original county name:

In [ ]:
cases_df['CountyName'] = cases_df['County'] + " County, Michigan"

cases_df

Great! But if we look a little further, we'll notice that Detroit's `CountyName` is now "Detroit County, Michigan" which isn't right. It's actually part of Wayne County, so let's fix that. We can update that row + column by finding the index of the data frame and using the following code:

In [ ]:
# update the CountyName for index 15

cases_df.at[15, "CountyName"] = "Wayne County, Michigan"

In [ ]:
cases_df

Now we can merge the two data frames on the column called `CountyName` with the following. We'll save the merged data in a data frame called `cases_population_df`:

In [ ]:
# merge 
cases_population_df = cases_df.merge(michigan_population_df, on='CountyName')

cases_population_df

Now, let's sort the combined data on Population and Cases (descending) to see if the counties with biggest population have the most cases. How does the data look? Anything stand out?

In [ ]:
cases_population_df.sort_values(['Population', 'Cases'], ascending=False)

In [ ]:
# if we want to re-order the columns we can do something like this:
cases_population_df = cases_population_df[['County','CountyName','Population','Cases','Reported Deaths','StateFIPS','CountyFIPS']]


In [ ]:
# and let's sort it again
cases_population_df.sort_values(['Population', 'Cases'], ascending=False)

### This is a start - what should we do next?

Ok, so this is a simple start to looking at some of the county level covid data alongside other data sets (in this case it was simply the county population from the Census). There is a bunch of Census data that we can look at along side this (sex, age, race, how people commute, etc). Of you can look at the BLS, [Detroit's Data Portal](https://data.detroitmi.gov/) or a number of other places. How should we help our friends at the Detroit Free Press? 